## Stations Map Parsing

Data was scraped from this source https://www.aviationweather.gov/docs/metar/stations.txt 

Logic to generate a CSV per location + date combo inside this text file

In [0]:
import json
import pandas 
import numpy
import os
import re
import csv
import pandas as pd

### Weather Station Map Guideline

`
!   CD = 2 letter state (province) abbreviation
!   STATION = 16 character station long name
!   ICAO = 4-character international id
!   IATA = 3-character (FAA) id
!   SYNOP = 5-digit international synoptic number
!   LAT = Latitude (degrees minutes)
!   LON = Longitude (degree minutes)
!   ELEV = Station elevation (meters)
!   M = METAR reporting station.   Also Z=obsolete? site
!   N = NEXRAD (WSR-88D) Radar site
!   V = Aviation-specific flag (V=AIRMET/SIGMET end point, A=ARTCC T=TAF U=T+V)
!   U = Upper air (rawinsonde=X) or Wind Profiler (W) site
!   A = Auto (A=ASOS, W=AWOS, M=Meso, H=Human, G=Augmented) (H/G not yet impl.)
!   C = Office type F=WFO/R=RFC/C=NCEP Center
!   Digit that follows is a priority for plotting (0=highest)
!   Country code (2-char) is last column`


**Total of 16 different fields**

In [0]:
!ls

Airline Delays and Weather - Project data.py
Archive.zip
README.md
Screen Shot 2020-10-28 at 11.26.34 AM.png
Stations Map Parsing.ipynb
airline-delays-literature
data_chunks_cleaned
myfile.txt
station_csvs
station_maps

### Just exploring here

In [0]:
# Using readlines() 
station_maps = open('station_maps/station_map.txt', 'r') 
lines = station_maps.readlines() 
  
count = 0

# Strips the newline character 
for line in lines:
    print("Line{}: {}".format(count, line.strip())) 

Line0: ALASKA 19-DEC-18
Line0: CD STATION ICAO IATA SYNOP LAT LONG ELEV M N V U A C
Line0: AK ADAK NAS PADK ADK 70454 51 53N 176 39W 3 X T 7 US
Line0: AK AKHIOK PAKH AKK 56 56N 154 11W 15 X 8 US
Line0: AK AKUTAN PAUT 54 08N 165 36W 25 X 7 US
Line0: AK ALASKAN ARTCC PZ74 Z74 60 09N 164 17W 1 A 8 US
Line0: AK ALASKAN ARTCC PZ76 Z76 65 15N 134 39W 647 A 8 US
Line0: AK AMBLER PAFM AFM 67 06N 157 51W 79 X 7 US
Line0: AK ANAKTUVUK PASS PAKP AKP 68 08N 151 44W 649 X 7 US
Line0: AK ANCHORAGE/ARTCC PZAN ZAN 61 10N 149 59W 22 A 8 US
Line0: AK ANCHORAGE INTL PANC ANC 70273 61 10N 150 01W 39 X T X A 5 US
Line0: AK ANCHORAGE/LAKE H PALH LHD 61 11N 149 58W 26 X A 7 US
Line0: AK ANCHORAGE/MERRIL PAMR MRI 61 13N 149 51W 43 X A 7 US
Line0: AK ANCHORAGE/WFO PAFC AFC 61 10N 150 02W 48 F 8 US
Line0: AK ANCHORAG/NIKISKI PAHG AHG 60 44N 151 21W 74 X 8 US
Line0: AK ANGOON SEAPLANE PAGN 70383 57 30N 134 35W 2 X 8 US
Line0: AK ANIAK PANI ANI 70232 61 35N 159 32W 21 X 7 US
Line0: AK ANNETTE ISLAND PANT ANN 70398 55 02N 131 34W 36 X X A 5 US
Line0: AK ANVIK PANV ANV 62 39N 160 11W 91 X 7 US
Line0: AK ARCTIC VILLAGE PARC ARC 68 07N 145 35W 626 X 7 US
Line0: AK ATKA PAAK AKA 52 13N 174 12W 11 X 7 US
Line0: AK ATQASUK BURNELL PATQ ATK 70 28N 157 26W 28 X 7 US
Line0: AK BARROW ARM-NSA 70027 71 19N 156 37W 7 X 8 US
Line0: AK BARTER ISLAND PABA BTI 70 08N 143 35W 1 X W 7 US
Line0: AK BETHEL/88D PABC ABC 60 48N 161 53W 49 X 8 US
Line0: AK BETHEL PABE BET 70219 60 47N 161 50W 28 X T X A 5 US
Line0: AK BETTLES PABT BTT 70174 66 55N 151 31W 195 X T A 6 US
Line0: AK BIG RIVER LAKES PALV LVR 60 49N 152 18W 12 X 7 US
Line0: AK BIRCHWOOD PABV BCV 61 25N 149 31W 21 X 7 US
Line0: AK BREVIG MISSION PFKT 65 20N 166 28W 8 X 7 US
Line0: AK BRYANT ARMY AF PAFR 70270 61 16N 149 39W 106 X 6 US
Line0: AK BUCKLAND PABL BVK 65 59N 161 09W 7 X 7 US
Line0: AK CANTWELL PATW TTW 63 23N 148 57W 668 X 7 US
Line0: AK CAPE LISBURNE PALU LUR 70104 68 53N 166 06W 8 X T W 6 US
Line0: AK CAPE NEWENHAM PAEH EHM 70305 58 39N 162 04W 101 X T 6 US
Line0: AK CAPE ROMANZOF PACZ CZF 70212 61 47N 166 02W 129 X T 6 US
Line0: AK CENTRAL AK PROF CEN 70197 65 30N 144 41W 259 W 8 US
Line0: AK CENTRAL PACE 65 34N 144 47W 286 X 7 US
Line0: AK CENTRAL PARL 65 34N 144 47W 284 X 7 US
Line0: AK CHANDALAR LAKE PALR WCR 67 30N 148 29W 585 X 7 US
Line0: AK CHEVAK PAVA 61 32N 165 36W 14 X 7 US
Line0: AK CHIGNIK BAY PAJC AJC 56 19N 158 22W 10 X 7 US
Line0: AK CHULITNA PAEC AEC 62 53N 149 34W 389 X 7 US
Line0: AK CIRCLE/PAFC RFC PACR CRC 65 50N 144 04W 182 X R 7 US
Line0: AK CLARKS POINT PFCL 58 50N 158 32W 23 X 7 US
Line0: AK COLD BAY PACD CDB 70316 55 13N 162 43W 23 X T X A 5 US
Line0: AK CORDOVA PACV CDV 70296 60 30N 145 28W 15 X T A 6 US
Line0: AK DEADHORSE PASC SCC 70 12N 148 28W 17 X T A 6 US
Line0: AK DEERING PADE DEE 66 04N 162 46W 5 X A 7 US
Line0: AK DELTA JUNCTION PABI BIG 70267 64 00N 145 43W 388 X T A 6 US
Line0: AK DILLINGHAM PADL DLG 70321 59 03N 158 30W 20 X T 6 US
Line0: AK DUTCH HARBOR PADU DUT 70489 53 54N 166 32W 4 X T 6 US
Line0: AK EAGLE PAEG EAA 64 47N 141 09W 273 X A 7 US
Line0: AK EARECKSON/SHEMYA PASY SYA 70414 52 43N 174 07E 6 X T X 5 US
Line0: AK EGEGIK PAII EII 58 11N 157 23W 31 X W 7 US
Line0: AK EIELSON AFB PAEI EIL 70265 64 40N 147 06W 166 X T 6 US
Line0: AK ELFIN COVE PAEL ELV 58 12N 136 21W 8 X 7 US
Line0: AK ELIM APRT PFEL 64 37N 162 16W 50 X 7 US
Line0: AK ELMENDORF AFB PAED EDF 70272 61 15N 149 50W 54 X T 6 US
Line0: AK EMMONAK PAEM ENM 62 47N 164 29W 2 X 7 US
Line0: AK EUREKA PAZK AZK 61 56N 147 10W 1002 X 7 US
Line0: AK FAIRBANKS PAFA FAI 70261 64 48N 147 52W 130 X T X A 5 US
Line0: AK FAIRBANKS/WFO PAFG AFG 64 48N 147 53W 140 F 8 US
Line0: AK FALSE PASS PAKF 54 51N 163 25W 6 X 7 US
Line0: AK FAREWELL LAKE PAFK FKK 70248 62 32N 153 36W 321 X 7 US
Line0: AK FORT WAINWRIGHT PAFB FBK 64 50N 147 37W 137 X T 6 US
Line0: AK FORT YUKON PAFY 66 34N 145 16W 131 Z 7 US
Line0: AK FORT YUKON PFYU FYU 70194 66 34N 145 14W 130 X 7 US
Line0: AK FRBNKS/PEDRO DOM PAPD APD 65 02N 147 30W 790 X 

Line0: VA KENBRIDGE KW31 W31 36 57N 078 11W 159 X 8 US
Line0: VA LANGLEY AFB/HAMP KLFI LFI 74598 37 05N 076 20W 2 X T 6 US
Line0: VA LAWRENCEVILLE KLVL LVL 36 47N 077 48W 98 X 6 US
Line0: VA LEESBURG/GODFREY KJYO JYO 39 04N 077 34W 112 X 7 US
Line0: VA LOUISA KLKU LKU 38 01N 077 58W 146 X 7 US
Line0: VA LURAY KLUA LUA 38 40N 078 30W 273 X 7 US
Line0: VA LURAY KW45 W45 38 40N 078 30W 275 Z 8 US
Line0: VA LYNCHBURG KLYH LYH 72410 37 19N 079 12W 282 X U A 6 US
Line0: VA MANASSAS MUNI KHEF HEF 38 43N 077 31W 53 X W 7 US
Line0: VA MARION / WYTHEVI KMKJ MKJ 36 53N 081 20W 777 X 7 US
Line0: VA MARTINSVILLE KMTV MTV 36 38N 080 01W 276 X 7 US
Line0: VA MELFA/ACCOMACK KMFV MFV 37 38N 075 46W 13 X 7 US
Line0: VA NEW MARKET K8W2 8W2 38 39N 078 42W 295 X 7 US
Line0: VA NEWPORT NEWS KPHF PHF 37 08N 076 30W 12 X T A 6 US
Line0: VA NORFOLK/HAMPTON KPVG PVG 36 47N 076 27W 7 X W 8 US
Line0: VA NORFOLK KORF ORF 72308 36 54N 076 12W 5 X U A 3 US
Line0: VA NORFOLK NAS/CHAM KNGU NGU 36 56N 076 16W 3 X T 6 US
Line0: VA NORFOLK/WAKEFLD KAKQ AKQ 36 59N 077 00W 34 X X A F 5 US
Line0: VA OCEANA NAS/SOUCE KNTU NTU 36 49N 076 01W 6 X T 6 US
Line0: VA ORANGE KOMH OMH 38 15N 078 03W 136 X 7 US
Line0: VA PETERSBURG KPTB PTB 37 10N 077 31W 56 X W 7 US
Line0: VA QUANTICO MCAF KNYG NYG 38 30N 077 18W 2 X T 6 US
Line0: VA QUINTON KW96 W96 37 30N 077 07W 35 X 7 US
Line0: VA RICHLANDS K6V3 6V3 37 04N 081 48W 809 Z 8 US
Line0: VA RICHLANDS KJFZ JFZ 37 04N 081 48W 809 X W 7 US
Line0: VA RICHMOND/CHESTER KFCI FCI 37 24N 077 31W 63 X 8 US
Line0: VA RICHMOND/HANOVER KOFP OFP 37 42N 077 26W 63 X A 7 US
Line0: VA RICHMOND KRIC RIC 72401 37 31N 077 19W 51 X U A 4 US
Line0: VA ROANOKE/BLACKSBG KRNK RNK 72318 37 12N 080 24W 648 X F 8 US
Line0: VA ROANOKE/FLOYD CY KFCX FCX 37 01N 080 16W 874 X 8 US
Line0: VA ROANOKE KROA ROA 72411 37 19N 079 58W 349 X T A 3 US
Line0: VA SALUDA KW75 W75 37 36N 076 27W 8 X 7 US
Line0: VA SOUTH BOSTON KW78 W78 36 42N 078 51W 120 X 7 US
Line0: VA SOUTH HILL/MECKL KAVC AVC 36 41N 078 03W 119 X 7 US
Line0: VA STAFFORD REGNL KRMN RMN 38 24N 077 27W 50 X W 7 US
Line0: VA STAUNTON/SHENAND KSHD SHD 38 16N 078 54W 360 X 7 US
Line0: VA STERLING KLWX LWX 38 59N 077 29W 89 X F 8 US
Line0: VA SUFFOLK KSFQ SFQ 36 41N 076 36W 19 X 7 US
Line0: VA TANGIER ISLAND KTGI TGI 37 49N 075 59W 1 X 8 US
Line0: VA TAPPAHANNOCK KXSA XSA 37 52N 076 54W 41 X W 8 US
Line0: VA VOLENS KVQN VQN 36 57N 078 59W 280 Z 8 US
Line0: VA WALLOPS ISLAND KWAL WAL 72402 37 56N 075 28W 10 X X A 5 US
Line0: VA WARRENTON KHWY HWY 38 35N 077 43W 93 X W 8 US
Line0: VA WARRENTON KW66 W66 38 35N 077 43W 103 Z 8 US
Line0: VA WASH DC/DULLES KIAD IAD 72403 38 56N 077 27W 90 X T X A 0 US
Line0: VA WAYNESBORO KW13 W13 38 05N 078 57W 440 X 7 US
Line0: VA WEST POINT KFYJ FYJ 37 31N 076 46W 3 X W 7 US
Line0: VA WILLIAMSBURG KJGG JGG 37 14N 076 43W 13 X W 7 US
Line0: VA WILLIAMSBURG KW94 W94 37 18N 076 38W 11 Z 8 US
Line0: VA WINCHESTER RGNL KOKV OKV 39 08N 078 09W 213 X 7 US
Line0: VA WISE/LONESOME PI KLNP LNP 36 59N 082 31W 817 X 7 US
Line0: 
Line0: VERMONT 30-APR-13
Line0: CD STATION ICAO IATA SYNOP LAT LONG ELEV M N V U A C
Line0: VT BALL MOUNTAIN K8B3 8B3 43 07N 072 48W 345 Z 8 US
Line0: VT BENNINGTON K5B5 5B5 42 53N 073 15W 244 Z 8 US
Line0: VT BENNINGTON KDDH DDH 42 54N 073 15W 241 X A 7 US
Line0: VT BURLINGTON KBTV BTV 72617 44 28N 073 09W 101 X T A F 3 US
Line0: VT CANAAN KVT1 VT1 44 59N 071 30W 317 Z 8 US
Line0: VT COLCHESTER/88D KCXX CXX 44 31N 073 10W 97 X 8 US
Line0: VT HIGHGATE KFSO FSO 44 56N 073 06W 70 X W 8 US
Line0: VT JAY PEAK KJAY JAY 44 56N 072 32W 572 Z 8 US
Line0: VT LYNDONVILLE K6B8 6B8 44 34N 072 01W 363 Z 8 US
Line0: VT LYNDONVILLE KCDA CDA 44 34N 072 01W 357 X W 8 US
Line0: VT MIDDLEBURY K6B0 6B0 43 59N 073 06W 149 X W 8 US
Line0: VT MONTPELIER/BARRE KMPV MPV 44 12N 072 34W 336 X U A 4 US
Line0: VT MORRISVILLE KMVL MVL 44 32N 072 37W 226 X 7 US
Line0: VT NEWPORT STATE KEFK EFK 44 53N 072 14W 282 X W 8 US
Line0: VT POWNAL KPOW POW 42 47N 073 13W 348 Z 8 US
Line0: VT RUTLAND STA

Line0: INNSBRUCK AIRPOR LOWI 11120 47 16N 011 20E 570 X T X 6 AT
Line0: KLAGENFURT(CIV/M LOWK 11231 46 39N 014 19E 450 X T 6 AT
Line0: KREMS-LANGENLOIS LOAG 48 27N 015 38E 312 X 7 AT
Line0: LANGENLEBARN/TUL LOXT 11030 48 19N 016 07E 176 X T 7 AT
Line0: LINZ (CIV/MIL) LOWL 11010 48 13N 014 11E 290 X T X 6 AT
Line0: OBEROSTERREICH LOLF 48 31N 014 25E 685 X 7 AT
Line0: PUNITZ/GUESSING LOGG 11195 47 09N 016 19E 290 X 7 AT
Line0: SALZBURG AIRPORT LOWS 11150 47 47N 013 01E 427 X T 6 AT
Line0: SCHWAZ (HELIPORT LOXS 11122 47 19N 011 41E 543 X 7 AT
Line0: VIENNA/SCHWECHAT LOWW 11036 48 07N 016 34E 179 X T 6 AT
Line0: VOESLAU-KOTTINGB LOAV 11181 47 58N 016 16E 232 X T 6 AT
Line0: WIEN/HOHE WARTE 11035 48 15N 016 22E 200 X 7 AT
Line0: WR.NEUSTADT EAST LOAN 47 51N 016 15E 266 X T 7 AT
Line0: WR.NEUSTADT WEST LOXN 11182 47 50N 016 13E 280 X T 6 AT
Line0: ZELL AM SEE LOWZ 11143 47 18N 012 47E 754 X T 6 AT
Line0: ZELTWEG (MIL) LOXZ 11165 47 12N 014 45E 678 X T 7 AT
Line0: 
Line0: PORTUGAL 11-SEP-15
Line0: ALVERCA LPAR 38 53N 009 02W 2 X T 6 PT
Line0: BEJA (POR/GER-AF LPBJ 38 05N 007 56W 196 X T 7 PT
Line0: BRAGANCA LPBG 08575 41 51N 006 42W 692 X 7 PT
Line0: CASCAIS LPCS 38 44N 009 21W 99 X T 7 PT
Line0: CORVO IS AZORES LPCR 39 40N 031 07W 26 X 8 PT
Line0: FARO LPFR 37 01N 007 58W 3 X T 6 PT
Line0: FLORES ISLAND LPFL 39 27N 031 07W 35 X 7 PT
Line0: FUNCHAL 08522 32 38N 016 54W 56 X 7 PT
Line0: FUNCHAL/MADEIRA LPFU 08521 32 40N 016 46W 55 Z 8 PT
Line0: FUNCHAL/MADEIRA LPMA 32 42N 016 46W 54 X T 6 PT
Line0: GRACIOSA IS AZOR LPGR 39 06N 028 02W 26 X 8 PT
Line0: HORTA 08506 38 31N 028 38W 62 X 7 PT
Line0: HORTA/FAIAL ISLA LPHR 38 31N 028 43W 39 X T 6 PT
Line0: LAJES AB LPLA 38 46N 027 06W 60 X T X 6 PT
Line0: LAJES/SANTA RITA 08508 38 45N 027 05W 55 X 7 PT
Line0: LISBON/GAGO C 08579 38 46N 009 08W 105 X 7 PT
Line0: LISBON/PORTELA LPPT 38 46N 009 08W 98 X T 6 PT
Line0: MONTE REAL(POR-A LPMR 39 49N 008 53W 50 X T 7 PT
Line0: MONTIJO (POR-AFB LPMT 38 42N 009 03W 4 X T 7 PT
Line0: OTA (POR-AFB) LPOT 08539 39 07N 008 58W 49 X 7 PT
Line0: OVAR MIL LPOV 40 55N 008 39W 16 X T 6 PT
Line0: PONTA DELGADA/NO LPPD 37 44N 025 41W 62 X T 6 PT
Line0: PORTO/PEDRAS RUB LPPR 41 14N 008 41W 64 X T 6 PT
Line0: PORTO SANTO ISLA LPPS 33 04N 016 21W 89 X T 6 PT
Line0: RPICO IS AZORES LPPI 38 33N 028 26W 33 X 8 PT
Line0: SANTA MARIA ISLA LPAZ 36 58N 025 10W 96 X T 6 PT
Line0: SINTRA/GRANJA (A LPST 38 49N 009 20W 129 X T 7 PT
Line0: TANCOS (POR-AFB) LPTN 08552 39 28N 008 25W 80 X 7 PT
Line0: VILA REAL LPVR 08567 41 16N 007 43W 562 X 7 PT
Line0: 
Line0: SERBIA 20-MAR-09
Line0: BATAJNICA LYBT 44 56N 020 15E 78 X T 8 CS
Line0: BELGRADE/SURCIN LYBE 13272 44 49N 020 17E 93 X T 6 CS
Line0: BEOGRAD KOSUTNJA 13275 44 47N 020 32E 243 X 7 CS
Line0: KRALJEVO LYKV 43 49N 020 35E 209 X T 8 CS
Line0: MOSTAR LYMO 43 21N 017 48E 108 X 7 CS
Line0: NIS LYNI 13388 43 20N 021 51E 197 X T 6 CS
Line0: OHRID LYOH 41 07N 020 48E 761 Z 7 CS
Line0: PODGORICA(TITOGR LYPG 42 22N 019 15E 32 X T 6 CS
Line0: PRISHTINA LYPR 42 35N 021 02E 546 Z T 8 CS
Line0: TITOGRAD/GOLUBOV LYTI 13462 42 22N 019 15E 33 X 7 CS
Line0: TIVAT LYTV 13457 42 24N 018 43E 3 X T 6 CS
Line0: UZICE-PONIKVE LYUZ 43 54N 019 40E 914 X T 8 CS
Line0: VRSAC LYVR 13183 45 08N 021 19E 82 X T 7 CS
Line0: 
Line0: KOSOVO 05-JUL-11
Line0: PRISHTINA BKPR 42 35N 021 02E 545 X T 6 CS
Line0: 
Line0: BOSNIA-HERZEGOVINA 05-JUL-11
Line0: BANJA LUKA LQBK 13242 44 56N 017 18E 114 X T 8 CS
Line0: MOSTAR AIRPORT LQMO 13348 43 17N 017 50E 49 X T 8 BA
Line0: SARAJEVO/BUTMIR LQSA 13353 43 49N 018 19E 505 X T 6 CS
Line0: TUZLA LQTZ 13257 44 28N 018 44E 242 X T 8 CS
Line0: 
Line0: ROMANIA 02-JAN-19
Line0: ARAD LRAR 15200 46 11N 021 16E 102 X T 6 RO
Line0: AVRAMENI 15001 48 01N 026 57E 240 X 8 RO
Line0: BACAU LRBC 15150 46 31N 026 55E 183 X T 7 RO
Line0: BAIA MARE LRBM 15014 47 40N 023 28E 180 X T 7 RO
Line0: BOBOC LRBO 45 13N 026 58E 104 T 7 RO
Line0: BORCEA FETESTI LRFT 44 23N 027 43E 54 T 6 RO
Line0: BUCHAREST/BANEAS LRBS 15420 44 30N 026 05E 91 X T X 6 

### Approach

Was gonna try something fancy, with regex, but every line has a strict number of bytes, so we can just use that to delimit. 

There are 4 different lengths of lines



| Line Typ | Length      |
|-----|---------------|
| Place + Date   | 29 |
| Header   | 79      |
| Data Row   | 84 |
| NewLineCharacter   | 1 |

In [0]:
print("Example 1")
print()
print(len(lines[0]))
print(lines[0])
print()
print(len(lines[1]))
print(lines[1])
print()
print(len(lines[2]))
print(lines[2])

print()
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
print()

print("Example 2")
print()

print(len(lines[272])) #This is a new line character
print(lines[272])
print()
print(len(lines[273]))
print(lines[273])
print()
print(len(lines[274]))
print(lines[274])
print(len(lines[275]))
print(lines[275])

Example 1

29
ALASKA 19-DEC-18


79
CD STATION ICAO IATA SYNOP LAT LONG ELEV M N V U A C


84
AK ADAK NAS PADK ADK 70454 51 53N 176 39W 3 X T 7 US


~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

Example 2

1



29
ARKANSAS 27-JUN-19


79
CD STATION ICAO IATA SYNOP LAT LONG ELEV M N V U A C

84
AR ARKADELPHIA KADF ADF 34 06N 093 04W 54 X T 8 US

### These are all the uppper bound index slots for each field in a given line.

* WEATHER_REV - is treated a bit differently
* PRIORITY - Reverse Indexed
* COUNTRY - Reverse Indexed

In [0]:
#LEN 29
PLACE_i = 19
DATE_i = 29

#LEN 84
CD_i = 2
STATION_i = 20
ICAO_i = 26
IATA_i = 32
SYNOP_i = 39
LAT_i = 47
LONG_i = 57
ELEV_i = 62
WEATHER_REV = 84-ELEV_i

### The Parsing!

Function that reads in lines from .txt file that we've set and generates CSV's for each table,

Total **256** tables, therefore 256 CSV's get generated

In [0]:
#If you pass verbose => True, print debug as the loop runs
def parse_tables(lines, verbose=False):
    
    ####
    #Just for sanity, these values aren't used for anything
    tot_tables = 0
    tot_headers = 0  
    ####
    
    cur_table_name = ""
    cur_table_rows = []
    us_table_names = set()
    
    for cur_line_num, line in enumerate(lines, 1):
        
        #New Table Name
        if len(line) == 29:
            PLACE = re.sub("/","-", line[:PLACE_i].strip())
            PLACE = re.sub(" ", "_", PLACE)
            DATE = re.sub("-","_",line[PLACE_i:DATE_i].strip())
            cur_table_name = PLACE + "-" + DATE
            
            if verbose:
                print(cur_table_name)
                
            tot_tables+=1
            
        #Table Header
        elif len(line) == 79:
            headers = re.sub('\s{2,}',',',line.strip()).split(",")
            headers.extend(["PRIOR", "COUNTRY"])
            cur_table_rows.append(headers)
            tot_headers+=1
            
            if verbose:
                print(headers)
            
            
        #Row Value
        elif len(line) == 84:
            
            CD = line[0:CD_i].strip()
            STATION = re.sub("/","-",line[CD_i:STATION_i].strip())
            ICAO = line[STATION_i:ICAO_i].strip()
            IATA = line[ICAO_i:IATA_i].strip()
            SYNOP = line[IATA_i:SYNOP_i].strip()
            LAT = line[SYNOP_i:LAT_i].strip()
            LONG = line[LAT_i:LONG_i].strip()
            ELEV = line[LONG_i:ELEV_i].strip()
         
            WEATHER_STUFF = line[-WEATHER_REV:-6]
            M, N, V, U, A, C= [WEATHER_STUFF[i:i+3].strip() for i in range(0, len(WEATHER_STUFF), 3)]
               
            PRIOR = line[-5:-3].strip()          
            COUNTRY = line[-3:].strip()
            
            ROW = [CD,STATION,ICAO,IATA,SYNOP,LAT,LONG,ELEV,M,N,V,U,A,C,PRIOR,COUNTRY]

            if verbose:
                print(ROW)

            #Add Row to tracker
            cur_table_rows.append(ROW)
            
            #Not most efficient but works LOL
            if COUNTRY == "US":
                us_table_names.add(cur_table_name)
        
        #New Line character, indicates we are about to start on a new table
        else:

            if verbose:
                print()
                print()
                
            #Just ensuring that we have all the correct number of columns or else something is wrong!
            good = True
            for row in cur_table_rows:
                if len(row) != 16:
                    good = False
                    print(f"SOMETHING IS WRONG - {cur_table_name}")
                    break
            
            #If good then make the CSV!
            if good:
                print(f"GOOD TO MAKE CSV - {cur_table_name}")
                print()
                
                name = cur_table_name
                if cur_table_name in us_table_names:
                    name = "US_STATIONS/"+cur_table_name

                #Make this path if it doesn't exist
                if not os.path.exists('station_csvs'):
                    os.makedirs('station_csvs')
                    os.makedirs('station_csvs/US_STATIONS')                    
                    
                #Write all the rows of current table to a CSV file
                with open(f'station_csvs/{name}.csv', 'w', newline='') as file:
                    writer = csv.writer(file)
                    for liner in cur_table_rows:
                        writer.writerow(liner)
                
                cur_table_name = ""
                cur_table_rows = []             

                
    print("tot_headers",tot_headers)
    print("tot_tables",tot_tables)    

### Generate The CSV's

In [0]:
parse_tables(lines, True)

ALASKA-19_DEC_18
['CD', 'STATION', 'ICAO', 'IATA', 'SYNOP', 'LAT', 'LONG', 'ELEV', 'M', 'N', 'V', 'U', 'A', 'C', 'PRIOR', 'COUNTRY']
['AK', 'ADAK NAS', 'PADK', 'ADK', '70454', '51 53N', '176 39W', '3', 'X', '', 'T', '', '', '', '7', 'US']
['AK', 'AKHIOK', 'PAKH', 'AKK', '', '56 56N', '154 11W', '15', 'X', '', '', '', '', '', '8', 'US']
['AK', 'AKUTAN', 'PAUT', '', '', '54 08N', '165 36W', '25', 'X', '', '', '', '', '', '7', 'US']
['AK', 'ALASKAN ARTCC', 'PZ74', 'Z74', '', '60 09N', '164 17W', '1', '', '', 'A', '', '', '', '8', 'US']
['AK', 'ALASKAN ARTCC', 'PZ76', 'Z76', '', '65 15N', '134 39W 6', '47', '', '', 'A', '', '', '', '8', 'US']
['AK', 'AMBLER', 'PAFM', 'AFM', '', '67 06N', '157 51W', '79', 'X', '', '', '', '', '', '7', 'US']
['AK', 'ANAKTUVUK PASS', 'PAKP', 'AKP', '', '68 08N', '151 44W 6', '49', 'X', '', '', '', '', '', '7', 'US']
['AK', 'ANCHORAGE-ARTCC', 'PZAN', 'ZAN', '', '61 10N', '149 59W', '22', '', '', 'A', '', '', '', '8', 'US']
['AK', 'ANCHORAGE INTL', 'PANC', 'ANC', '70273', '61 10N', '150 01W', '39', 'X', '', 'T', 'X', 'A', '', '5', 'US']
['AK', 'ANCHORAGE-LAKE H', 'PALH', 'LHD', '', '61 11N', '149 58W', '26', 'X', '', '', '', 'A', '', '7', 'US']
['AK', 'ANCHORAGE-MERRIL', 'PAMR', 'MRI', '', '61 13N', '149 51W', '43', 'X', '', '', '', 'A', '', '7', 'US']
['AK', 'ANCHORAGE-WFO', 'PAFC', 'AFC', '', '61 10N', '150 02W', '48', '', '', '', '', '', 'F', '8', 'US']
['AK', 'ANCHORAG-NIKISKI', 'PAHG', 'AHG', '', '60 44N', '151 21W', '74', '', 'X', '', '', '', '', '8', 'US']
['AK', 'ANGOON SEAPLANE', 'PAGN', '', '70383', '57 30N', '134 35W', '2', 'X', '', '', '', '', '', '8', 'US']
['AK', 'ANIAK', 'PANI', 'ANI', '70232', '61 35N', '159 32W', '21', 'X', '', '', '', '', '', '7', 'US']
['AK', 'ANNETTE ISLAND', 'PANT', 'ANN', '70398', '55 02N', '131 34W', '36', 'X', '', '', 'X', 'A', '', '5', 'US']
['AK', 'ANVIK', 'PANV', 'ANV', '', '62 39N', '160 11W', '91', 'X', '', '', '', '', '', '7', 'US']
['AK', 'ARCTIC VILLAGE', 'PARC', 'ARC', '', '68 07N', '145 35W 6', '26', 'X', '', '', '', '', '', '7', 'US']
['AK', 'ATKA', 'PAAK', 'AKA', '', '52 13N', '174 12W', '11', 'X', '', '', '', '', '', '7', 'US']
['AK', 'ATQASUK BURNELL', 'PATQ', 'ATK', '', '70 28N', '157 26W', '28', 'X', '', '', '', '', '', '7', 'US']
['AK', 'BARROW ARM-NSA', '', '', '70027', '71 19N', '156 37W', '7', '', '', '', 'X', '', '', '8', 'US']
['AK', 'BARTER ISLAND', 'PABA', 'BTI', '', '70 08N', '143 35W', '1', 'X', '', '', '', 'W', '', '7', 'US']
['AK', 'BETHEL-88D', 'PABC', 'ABC', '', '60 48N', '161 53W', '49', '', 'X', '', '', '', '', '8', 'US']
['AK', 'BETHEL', 'PABE', 'BET', '70219', '60 47N', '161 50W', '28', 'X', '', 'T', 'X', 'A', '', '5', 'US']
['AK', 'BETTLES', 'PABT', 'BTT', '70174', '66 55N', '151 31W 1', '95', 'X', '', 'T', '', 'A', '', '6', 'US']
['AK', 'BIG RIVER LAKES', 'PALV', 'LVR', '', '60 49N', '152 18W', '12', 'X', '', '', '', '', '', '7', 'US']
['AK', 'BIRCHWOOD', 'PABV', 'BCV', '', '61 25N', '149 31W', '21', 'X', '', '', '', '', '', '7', 'US']
['AK', 'BREVIG MISSION', 'PFKT', '', '', '65 20N', '166 28W', '8', 'X', '', '', '', '', '', '7', 'US']
['AK', 'BRYANT ARMY AF', 'PAFR', '', '70270', '61 16N', '149 39W 1', '06', 'X', '', '', '', '', '', '6', 'US']
['AK', 'BUCKLAND', 'PABL', 'BVK', '', '65 59N', '161 09W', '7', 'X', '', '', '', '', '', '7', 'US']
['AK', 'CANTWELL', 'PATW', 'TTW', '', '63 23N', '148 57W 6', '68', 'X', '', '', '', '', '', '7', 'US']
['AK', 'CAPE LISBURNE', 'PALU', 'LUR', '70104', '68 53N', '166 06W', '8', 'X', '', 'T', '', 'W', '', '6', 'US']
['AK', 'CAPE NEWENHAM', 'PAEH', 'EHM', '70305', '58 39N', '162 04W 1', '01', 'X', '', 'T', '', '', '', '6', 'US']
['AK', 'CAPE ROMANZOF', 'PACZ', 'CZF', '70212', '61 47N', '166 02W 1', '29', 'X', '', 'T', '', '', '', '6', 'US']
['AK', 'CENTRAL AK PROF', '', 'CEN', '70197', '65 30N', '144 41W 2', '59', '', '', '', 'W', '', '', '8', 'US']
['AK', 'CENTRAL', 'PACE', '', '', '65 34N', '144 47W 2', '86', 'X', '', '', '', '', '', '7', 'US']
['AK', 'CENTRAL', 'PARL', '', '', '65 34N',

NEW_HAMPSHIRE-26_APR_06
['CD', 'STATION', 'ICAO', 'IATA', 'SYNOP', 'LAT', 'LONG', 'ELEV', 'M', 'N', 'V', 'U', 'A', 'C', 'PRIOR', 'COUNTRY']
['NH', 'BERLIN', 'KBML', 'BML', '72616', '44 35N', '071 11W 3', '44', 'X', '', '', '', 'A', '', '7', 'US']
['NH', 'CONCORD', 'KCON', 'CON', '72605', '43 12N', '071 30W 1', '02', 'X', '', 'U', '', 'A', '', '6', 'US']
['NH', 'DIAMOND LAKE', 'K8B2', '8B2', '', '44 57N', '071 19W 6', '78', 'Z', '', '', '', '', '', '8', 'US']
['NH', 'ISLE OF SHOALS', '', '', '99427', '42 58N', '070 22W', '13', 'Z', '', '', '', '', '', '8', 'US']
['NH', 'JAFFREY', 'KAFN', 'AFN', '', '42 48N', '072 00W 3', '08', 'X', '', '', '', 'A', '', '7', 'US']
['NH', 'KEENE-DILLANT', 'KEEN', 'EEN', '', '42 53N', '072 16W 1', '43', 'X', '', '', '', 'W', '', '7', 'US']
['NH', 'LACONIA MUNI', 'KLCI', 'LCI', '', '43 34N', '071 25W 1', '66', 'X', '', '', '', 'W', '', '7', 'US']
['NH', 'LEBANON', 'KLEB', 'LEB', '', '43 38N', '072 18W 1', '69', 'X', '', 'T', '', 'A', '', '6', 'US']
['NH', 'MANCHESTER', 'KMHT', 'MHT', '', '42 56N', '071 26W', '68', 'X', '', 'T', '', 'A', '', '6', 'US']
['NH', 'MOUNT WASHINGTON', 'KMWN', 'MWN', '72613', '44 16N', '071 18W 19', '11', 'X', '', '', '', '', '', '7', 'US']
['NH', 'NASHUA-ARTCC-BOS', 'KZBW', 'ZBW', '', '42 44N', '071 29W', '61', '', '', 'A', '', '', '', '8', 'US']
['NH', 'NASHUA-BOIRE FLD', 'KASH', 'ASH', '', '42 47N', '071 31W', '56', 'X', '', '', '', '', '', '7', 'US']
['NH', 'NORTH GROTON', 'KNGR', 'NGR', '', '43 46N', '071 52W 3', '90', 'Z', '', '', '', '', '', '8', 'US']
['NH', 'PLYMOUTH', 'K1P1', '1P1', '', '43 47N', '071 45W 1', '55', 'X', '', '', '', '', '', '7', 'US']
['NH', 'PORTSMOUTH-PEASE', 'KPSM', 'PSM', '', '43 05N', '070 49W', '34', 'X', '', 'T', '', '', '', '6', 'US']
['NH', 'ROCHESTER', 'K6B1', '6B1', '', '43 17N', '070 55W 1', '00', 'Z', '', '', '', '', '', '8', 'US']
['NH', 'ROCHESTER', 'KDAW', 'DAW', '', '43 17N', '070 55W 1', '00', 'X', '', '', '', 'A', '', '7', 'US']
['NH', 'TAMWORTH', 'KTMW', 'TMW', '', '43 54N', '071 18W 2', '41', 'Z', '', '', '', '', '', '8', 'US']
['NH', 'WHITEFIELD', 'KHIE', 'HIE', '', '44 22N', '071 33W 3', '21', 'X', '', 'T', '', 'A', '', '7', 'US']


GOOD TO MAKE CSV - NEW_HAMPSHIRE-26_APR_06

NEW_JERSEY-13_FEB_12
['CD', 'STATION', 'ICAO', 'IATA', 'SYNOP', 'LAT', 'LONG', 'ELEV', 'M', 'N', 'V', 'U', 'A', 'C', 'PRIOR', 'COUNTRY']
['NJ', 'ALEXANDRIA', 'KN85', 'N85', '', '40 35N', '075 01W 1', '46', 'X', '', '', '', 'W', '', '7', 'US']
['NJ', 'ANDOVER', 'K12N', '12N', '', '41 01N', '074 44W 1', '78', 'X', '', '', '', 'A', '', '7', 'US']
['NJ', 'ATLANTIC CITY', 'KACY', 'ACY', '72407', '39 27N', '074 34W', '18', 'X', '', 'T', '', 'A', '', '4', 'US']
['NJ', 'BELMAR-FARMDALE', 'KBLM', 'BLM', '', '40 10N', '074 07W', '45', 'X', '', '', '', '', '', '7', 'US']
['NJ', 'BLAIRSTOWN', 'K1N7', '1N7', '', '40 58N', '075 00W 1', '13', 'X', '', '', '', 'W', '', '7', 'US']
['NJ', 'CALDWELL', 'KCDW', 'CDW', '', '40 53N', '074 17W', '52', 'X', '', '', '', 'A', '', '7', 'US']
['NJ', 'COYLE VOR', 'KCYN', 'CYN', '', '39 49N', '074 26W', '70', '', '', 'V', '', '', '', '8', 'US']
['NJ', 'CROSS KEYS', 'K17N', '17N', '', '39 42N', '075 02W', '49', 'X', '', '', '', 'W', '', '7', 'US']
['NJ', 'HAMMONTON', 'KN81', 'N81', '', '39 40N', '074 45W', '20', 'X', '', '', '', 'W', '', '7', 'US']
['NJ', 'LAKEHURST NAS', 'KNEL', 'NEL', '72409', '40 02N', '074 20W', '26', 'X', '', '', '', '', '', '7', 'US']
['NJ', 'LAKEWOOD', 'KN12', 'N12', '', '40 04N', '074 11W', '13', 'X', '', '', '', 'W', '', '7', 'US']
['NJ', 'LINCOLN PARK', 'KN07', 'N07', '', '40 57N', '074 19W', '56', 'X', '', '', '', 'W', '', '7', 'US']
['NJ', 'LINDEN', 'KLDJ', 'LDJ', '', '40 37N', '074 15W', '6', 'X', '', '', '', 'W', '', '8', 'US']
['NJ', 'LUMBERTON', 'KN14', 'N14', '', '39 56N', '074 48W', '15', 'X', '', '', '', 'W', '', '7', 'US']
['NJ', 'MANCHESTER 88D', 'KDIX', 'DIX', '', '39 57N', '074 25W', '45', '', 'X', '', '', '', '', '8', 'US']
['NJ', 'MANVILLE', 'K47N', '47N', '', '40 31N', '074 36W', '26', 'X', '

GUAM-23_DEC_01
['GU', 'AGANA', 'PGUM', '', '91212', '13 29N', '144 48E', '74', 'X', '', 'T', '', 'A', '', '6', 'US']
['GU', 'MARIANA-ANDERSEN', 'PGUA', '', '91218', '13 34N', '144 55E 1', '59', 'X', 'X', 'T', '', '', 'F', '3', 'US']
['GU', 'PEIPEINIMARU-TIN', 'PGWT', '', '', '14 59N', '145 37E', '84', 'X', '', 'T', '', '', '', '6', 'US']
['GU', 'TAGUAC', '', '', '91217', '13 33N', '144 49E 1', '11', '', '', '', 'Z', '', '', '8', 'US']
['GU', 'TAGUAC', 'PGAC', '', '', '13 33N', '144 49E 1', '11', '', '', '', 'X', '', '', '8', 'US']
['UM', 'PAGAN-N. MARIANA', 'PAGP', '', '99451', '18 00N', '145 25E', '2', 'X', '', '', '', '', '', '7', 'US']
['UM', 'SABANETTAN-TINIA', 'PGNT', '', '', '15 00N', '145 37E', '80', 'X', '', '', '', '', '', '7', 'US']
['UM', 'WEST TINIAN', 'PGTI', '', '', '15 00N', '145 37E', '82', 'X', '', '', '', '', '', '7', 'US']


GOOD TO MAKE CSV - GUAM-23_DEC_01

JOHNSTON-WAKE-XMAS-30_APR_13
['UM', 'CHRISTMAS-CASSID', 'PLCH', '', '91490', '01 59N', '157 21W', '5', 'X', '', 'T', '', '', '', '6', 'US']
['UM', 'ENIWETOK AUX AF', 'PKMA', '', '91250', '11 20N', '162 20E', '5', 'X', '', '', '', '', '', '7', 'US']
['UM', 'FANNING ISLAND', 'PLFA', '', '91487', '03 51N', '159 22W', '5', 'X', '', '', '', '', '', '7', 'US']
['UM', 'JARVIS ISLAND (A', 'PLUR', '', '91901', '00 22S', '160 01W', '2', 'X', '', '', '', '', '', '7', 'US']
['UM', 'JOHNSTON ISLAND', 'PJON', '', '91275', '16 44N', '169 31W', '5', 'X', '', 'T', '', '', '', '6', 'US']
['UM', 'PALMYRA-COOPER(A', 'PLPA', '', '91385', '05 52N', '162 03W', '2', 'X', '', '', '', '', '', '7', 'US']
['UM', 'WAKE ISLAND AIRF', 'PWAK', '', '91245', '19 17N', '166 37E', '7', 'X', '', 'T', '', '', '', '6', 'US']


GOOD TO MAKE CSV - JOHNSTON-WAKE-XMAS-30_APR_13

MARSHALL_ISLANDS-30_APR_13
['', 'ENEWETAK-MARSHAL', 'ENIP', '', '99448', '11 22N', '162 22E', '2', 'X', '', '', '', '', '', '7', 'NO']
['MH', 'KWAJALEIN-BUCHOL', 'PKWA', '', '91366', '08 43N', '167 43E', '6', 'X', '', 'T', '', '', '', '6', 'US']
['MH', 'MAJURO ATOLL-NWS', 'PKMR', '', '91446', '07 05N', '171 23E', '7', 'X', '', '', '', '', 'F', '6', 'US']
['MH', 'MAJURO WSO AIRPO', 'PKMJ', '', '91376', '07 04N', '171 16E', '2', 'X', '', 'T', 'X', '', '', '5', 'US']
['MH', 'MILI-MARSHALL IS', 'MLIP', '', '99450', '06 00N', '172 00E', '0', 'X', '', '', '', '', '', '7', 'US']
['', 'UJAE ATOLL-MARSH', 'UJAP', '', '99449', '08 55N', '165 45E', '2', 'X', '', '', '', '', '', '7', 'RU']


GOOD TO MAKE CSV - MARSHALL_ISLANDS-30_APR_13

MICRONESIA-12_JUL_02
['', 'BABELTHUAP-KOROR', 'PTRO', '', '91408', '07 22N', '134 33E', '30', 'X', '', 'T', 'X', '', '', '5', 'FM']
['', 'KOROR PALAU-NWS', 'PTKR', '', '', '07 20N', '134 29E', '30', 'X', '', '', '', '', 'F', '5', 'FM']
['', 'KOSRAE CAROLINES', 'PTTK', '', '91355', '05 20N', '162 56E', '2', 'X', '', '', '', '', '', '7', 'FM']
['', 'KOSRAE ISLAND', 'PTSA', '', '91356', '05 21N', '162 57E', '3', 'X', '', 'T', '', '', '', '6', 'FM']
['', 'POHNPEI AIRPORT', 'PTPN', '', '91348', '06 59N', '158 13E', '3', 'X', '', 'T', 'Z', '', '', '6', 'FM']
['', 'POHNPEI NWS', 'PTTP', '', '', '06 58N', '158 13E', '36', 'X', '', '', 'X', '', '', '5', 'FM']
['', 'ROTA INTL AIRP', 'PGRO', '', '91221', '14 11N', '145 15E 1', '81', 'X', '', 'T', '', '', '', '6', 'FM']
['', 'SAIPAN-KOBLER', 'PGSN', '', '91232', '15 07N', '145 43E', '65', 'X', '', 'T', '', '', '', '6', 'FM']
['', 'WENO IS-CHUUK', 'PTKK', '', '91334', '07 28N', '151 51E', '3', 'X', '', 'T', 'X', '', '', '5', 'FM']
['', 'YAP ISLAND', 'PTYA', '', '91413', '09 29N', '138 05E', '4', 'X', '', 'T', 'X', '', '', '5', 'FM']


GOOD TO MAKE CSV - MICRONESIA-12_JUL_02

PUERTO_RICO-04_OCT_17
['PR', 'AGUADILLA-BORINQ', 'TJBQ', '', '78514', '18 30N', '067 08W', '69', 'X', '', 'T', '', '', '', '6', 'US']
['PR', 'FERN LUIS RIBAS', 'TJIG', '', '', '18 27N', '066 06W', '3', 'X', '', '', '', '', '', '8', 'US']
['PR', 'MAYAGUEZ-EUGENIO', 'TJMZ', '', '', '18 16N', '067 09W', '9', 'X', '', 'T', '', '', '', '6', 'US']
['PR', 'PONCE-MERCEDITA', 'TJPS', '', '', '18 01N'

SLOVAKIA-02_JAN_19
['', 'BRATISLAVA IVANK', 'LZIB', 'BTS', '11816', '48 10N', '017 13E 1', '29', 'X', '', 'T', '', '', '', '6', 'SK']
['', 'KAMENICA NAD CIR', 'LZKC', '', '11993', '48 56N', '021 59E 1', '77', 'X', '', '', '', '', '', '6', 'SK']
['', 'KOSICE BARCA', 'LZKZ', 'KSC', '', '48 40N', '021 14E 2', '27', 'X', '', 'T', '', '', '', '6', 'SK']
['', 'LUCENEC AIRPORT', 'LZLU', '', '', '48 20N', '019 44E 2', '00', 'X', '', '', '', '', '', '8', 'SK']
['', 'MALACKY', 'LZMC', '', '', '48 24N', '017 07E 2', '07', '', '', 'T', '', '', '', '7', 'SK']
['', 'NITRA', 'LZNI', '', '', '48 17N', '018 08E 1', '35', 'X', '', '', '', '', '', '7', 'SK']
['', 'PIESTANY', 'LZPP', 'PZY', '', '48 37N', '017 50E 1', '60', 'X', '', 'T', '', '', '', '7', 'SK']
['', 'POPRAD-GANOVCE', '', '', '11952', '49 02N', '020 19E 6', '93', '', '', '', 'X', '', '', '7', 'SK']
['', 'POPRAD-TATRY', 'LZTT', 'TAT', '11934', '49 04N', '020 15E 6', '93', 'X', '', 'T', '', '', '', '6', 'SK']
['', 'PRESOV', 'LZPW', '', '', '49 02N', '021 18E 3', '23', '', '', 'T', '', '', '', '7', 'SK']
['', 'PRIEVIDZA', 'LZPE', '', '', '48 46N', '018 35E 2', '60', 'X', '', '', '', '', '', '7', 'SK']
['', 'SLIAC', 'LZSL', 'SLD', '', '48 38N', '019 08E 3', '11', 'X', '', 'T', '', '', '', '7', 'SK']
['', 'ZILINA', 'LZZI', 'ILZ', '', '49 14N', '018 37E 3', '11', 'X', '', 'T', '', '', '', '7', 'SK']


GOOD TO MAKE CSV - SLOVAKIA-02_JAN_19

ISRAEL-27_JUN_19
['', 'BEER-SHEVA-TEYMA', 'LLBS', '', '40191', '31 17N', '034 43E 2', '00', 'X', '', '', '', '', '', '7', 'IL']
['', 'BEN-GURION(CIV-M', 'LLBG', '', '40180', '32 00N', '034 53E', '35', 'X', '', 'T', '', '', '', '6', 'IL']
['', 'BEN YA AKOV', 'LLIB', '', '', '32 59N', '035 34E 2', '68', 'X', '', 'T', '', '', '', '8', 'IL']
['', 'EILAT', 'LLER', '', '', '29 43N', '035 01E', '88', 'X', '', 'T', '', '', '', '6', 'IL']
['', 'ELAT-J. HOZMAN(A', 'LLET', '', '40199', '29 33N', '034 57E', '13', 'X', '', 'T', '', '', '', '6', 'IL']
['', 'HAIFA (ISR-AFB)', 'LLHA', '', '40155', '32 48N', '035 02E', '3', 'X', '', 'T', '', '', '', '7', 'IL']
['', 'HERZLIYA', 'LLHZ', '', '', '32 11N', '034 50E', '37', '', '', 'T', '', '', '', '7', 'IL']
['', 'JERUSALEM-ATAROT', 'LLJR', '', '40290', '31 52N', '035 13E 7', '59', 'X', '', 'T', '', '', '', '6', 'IL']
['', 'OVDA (ISR-AFB-CI', 'LLOV', '', '40198', '29 56N', '034 56E 4', '32', 'X', '', 'T', '', '', '', '6', 'IL']
['', 'SDE DOV AIRPORT', 'LLSD', '', '', '32 07N', '034 47E', '14', 'X', '', 'T', '', '', '', '8', 'IL']


GOOD TO MAKE CSV - ISRAEL-27_JUN_19

MALTA-27_DEC_01
['', 'LUQA-MALTA', 'LMML', '', '16597', '35 51N', '014 28E', '78', 'X', '', 'T', '', '', '', '6', 'MT']
['', 'MALTA ACC', 'LMMM', '', '', '35 53N', '014 31E', '95', 'X', '', '', '', '', '', '7', 'MT']


GOOD TO MAKE CSV - MALTA-27_DEC_01

AUSTRIA-08_MAY_06
['', 'AIGEN-ENNSTAL (M', 'LOXA', '', '11157', '47 31N', '014 07E 6', '49', 'X', '', 'T', '', '', '', '7', 'AT']
['', 'GRAZ (MIL-CIV)', 'LOWG', '', '11240', '47 00N', '015 26E 3', '37', 'X', '', 'T', 'X', '', '', '6', 'AT']
['', 'HOHENEMS-DORNBIR', 'LOIH', '', '', '47 23N', '009 42E 4', '18', 'X', '', 'T', '', '', '', '6', 'AT']
['', 'HORSCHING (AUS-A', 'LOXL', '', '11011', '48 13N', '014 10E 3', '13', 'X', '', '', '', '', '', '7', 'AT']
['', 'INNSBRUCK AIRPOR', 'LOWI', '', '11120', '47 16N', '011 20E 5', '70', 'X', '', 'T', 'X', '', '', '6', 'AT']
['', 'KLAGENFURT(CIV-M', 'LOWK', '', '11231', '46 39N', '014 19E 4', '50', 'X', '', 'T', '', '', '', '6', 'AT']
['', 'KREMS-LANGENLOIS', 'LOAG', '', '', '48 27N', '015 38E 3', '12', 'X', '', '', '', '', '', '7', 'AT']
['', 'LANGENLEBARN-TUL', 'LOXT', '', '11030', '48 19N', '016 07E 1', '76', 'X', '', 'T', '', '', '', '7', 'AT']
['', 'LINZ (CIV-MIL)', 'LOWL', '', '11010', '48 13N', '014 11E 2', '90', 'X', '', 'T', 'X', '', '', '6', 'AT']
['', 'OBEROSTERREICH', 'LOLF', '', '', '48 31N', '014 25E 6', '85', 'X', '', '', '', '', '', '7', 'AT']
['', 'PUNITZ-GUESSING', 'LOGG', '', '11195', '47 09N', '016 19E 2', '90', 'X', '', '', '', '', '', '7', 'AT']
['

['', 'TOTTORI AIRPORT', 'RJOR', '', '47794', '35 31N', '134 10E', '11', 'X', '', '', '', '', '', '7', 'JP']
['', 'TOYAMA AIRPORT', 'RJNT', '', '47707', '36 38N', '137 11E', '30', 'X', '', 'T', '', '', '', '6', 'JP']
['', 'TSUIKI (JASDF)', 'RJFZ', '', '47840', '33 40N', '131 03E', '20', 'X', '', '', '', '', '', '7', 'JP']
['', 'TSUSHIMA AIRPORT', 'RJDT', '', '47799', '34 17N', '129 19E', '61', 'X', '', '', '', '', '', '7', 'JP']
['', 'UTSUNOMIYA (JGSD', 'RJTU', '', '47692', '36 31N', '139 52E', '99', 'X', '', '', '', '', '', '7', 'JP']
['', 'WAKKANAI AIRPORT', 'RJCW', '', '47441', '45 24N', '141 48E', '11', 'X', '', '', '', '', '', '7', 'JP']
['', 'YAKUSHIMA ISLAND', 'RJFC', '', '47836', '30 22N', '130 40E', '38', 'X', '', '', '', '', '', '7', 'JP']
['', 'YAMAGATA(CIV-JGS', 'RJSC', '', '47553', '38 25N', '140 22E 1', '04', 'X', '', '', '', '', '', '7', 'JP']
['', 'YAMAGUCHI-UBE AR', 'RJDC', '', '47786', '33 55N', '131 16E', '5', 'X', '', '', '', '', '', '7', 'JP']
['', 'YAO (CIV-JGSDF)', 'RJOY', '', '47779', '34 36N', '135 36E', '10', 'X', '', '', '', '', '', '7', 'JP']
['', 'YOKOSUKA FWF', 'RJTX', '', '47696', '35 18N', '139 41E', '5', 'X', '', 'T', '', '', '', '6', 'JP']
['', 'YOKOTA (JASDF-US', 'RJTY', '', '47642', '35 45N', '139 20E 1', '36', 'X', '', 'T', '', '', '', '6', 'JP']
['', 'YONAGUNI AIRPORT', 'ROYN', '', '47911', '24 28N', '122 58E', '21', 'X', '', '', '', '', '', '7', 'JP']
['', 'YORON ISLAND', 'RORY', '', '', '27 03N', '128 24E', '13', 'X', '', '', '', '', '', '7', 'JP']


GOOD TO MAKE CSV - 

KOREA-11_SEP_15
['', 'ANDONG', 'RKTA', '', '47136', '36 32N', '128 43E 1', '42', 'X', '', '', '', '', '', '7', 'KR']
['', 'BUSAN-KIMHAE INT', 'RKPK', '', '47153', '35 10N', '128 56E', '3', 'X', '', 'T', '', '', '', '6', 'KR']
['', 'CAMP CASEY-TONGD', 'RKST', '', '', '37 55N', '127 03E', '60', 'X', '', '', '', '', '', '7', 'KR']
['', 'CAMP PAGE-CHUNCH', 'RKNC', '', '47104', '37 52N', '127 43E', '75', 'X', '', 'T', '', '', '', '6', 'KR']
['', 'CAMP STANLEY-H-2', 'RKSX', '', '', '37 43N', '127 05E', '71', 'X', '', 'T', '', '', '', '6', 'KR']
['', 'CAMP WALKER (H-8', 'RKTG', '', '', '35 51N', '128 36E', '75', 'X', '', '', '', '', '', '7', 'KR']
['', 'CHAJANG NI', 'RKSI', '', '', '37 28N', '126 27E', '7', 'X', '', 'T', '', '', '', '8', 'KR']
['', 'COMMAND POST TAN', 'RKSH', '', '', '37 31N', '126 58E', '73', 'X', '', '', '', '', '', '7', 'KR']
['', 'DONGHAE RADAR', 'RKSB', '', '47106', '37 30N', '129 08E', '37', 'X', '', '', '', '', '', '7', 'KR']
['', 'GWANGJU (KOR-AFB', 'RKJJ', '', '47158', '35 07N', '126 49E', '13', 'X', '', 'T', '', '', '', '6', 'KR']
['', 'HAE MI', 'RKTP', '', '', '36 42N', '126 29E', '13', 'X', '', 'T', '', '', '', '7', 'KR']
['', 'HAEMI', 'RKTZ', '', '', '36 23N', '126 20E', '31', 'X', '', '', '', '', '', '7', 'KR']
['', 'HOENGSONG (KOR-A', 'RKNH', '', '47118', '37 25N', '127 57E 1', '01', 'X', '', '', '', '', '', '7', 'KR']
['', 'JEJU INTL AIRPO', 'RKPC', '', '47182', '33 30N', '126 29E', '23', 'X', '', 'T', '', '', '', '6', 'KR']
['', 'JHUNJU (KOR-ARMY', 'RKJU', '', '', '35 52N', '127 07E', '29', 'X', '', '', '', '', '', '7', 'KR']
['', 'JINHAE (KOR-ARM', 'RKPE', '', '', '35 08N', '128 41E', '2', 'X', '', '', '', '', '', '7', 'KR']
['', 'JONGJU (KAFB-AR', 'RKTU', '', '47128', '36 42N', '127 30E', '46', 'X', '', 'T', '', '', '', '6', 'KR']
['', 'JUNGJU', 'RKTV', '', '', '37 04N', '127 52E', '59', 'X', '', '', '', '', '', '7', 'KR']
['', 'JUNG WON (ROK-AB', 'RKTI', '', '47125', '37 01N', '127 52E', '91', 'X', '', '', '', '', '', '7', 'KR']
['', 'KANGNUNG (KOR-AF', 'RKNN', '', '47107', '37 45N', '128 56E', '11', 'X', '', 'T', '', '', '', '6', 'KR']
['', 'KEO JIN', 'RKNO', '', '', '38 28N', '128 28E 1', '04', 'X', '', '', '', '', '', '7', 'KR']
['', 'KOON-NI RANGE', 'RKSN', '', '69010', '37 01N', '126 45E', '15', 'X', '', '', '', '', '', '7', 'KR']
['', 'KOTAR RANGE', 'RKNR', '', '', '37 06N', '128 54E 7', '92', 'X', '', '', '', '', '', '7', 'KR']
['', 'KUNSAN (US-KOR-A', 'RKJK', '', '47141', '35 54N', '1

### One FAT U.S. Stations Table

Figured it might make it easier to join if we just have one huge US table, and filter then join as needed, so now that we have all the individual CSV's figured might as well build one joined table

In [0]:
path = "station_csvs/US_STATIONS"
us_station_filenames = os.listdir(path)
print(len(us_station_filenames))
us_station_filenames


55

['MONTANA-23_MAY_20.csv',
 'PENNSYLVANIA-15_OCT_19.csv',
 'WASHINGTON-14_JUN_17.csv',
 'NEVADA-14_JUN_17.csv',
 'WEST_VIRGINIA-07_OCT_14.csv',
 'KENTUCKY-23_MAY_20.csv',
 'NEW_YORK-30_JUN_18.csv',
 'GUAM-23_DEC_01.csv',
 'SOUTH_DAKOTA-28_FEB_12.csv',
 'RHODE_ISLAND-16_DEC_98.csv',
 'ILLINOIS-09_FEB_18.csv',
 'HAWAII-30_APR_13.csv',
 'MICHIGAN-23_MAY_20.csv',
 'IOWA-02_FEB_11.csv',
 'MARYLAND-13_FEB_12.csv',
 'NEW_HAMPSHIRE-26_APR_06.csv',
 'COLORADO-23_MAY_20.csv',
 'ARKANSAS-27_JUN_19.csv',
 'INDIANA-14_JUJ_20.csv',
 'WASHINGTON_D.C.-25_SEP_00.csv',
 'MAINE-22_MAR_12.csv',
 'NEW_MEXICO-14_JUL_20.csv',
 'FLORIDA-14_JUL_20.csv',
 'MISSOURI-30_JUN_18.csv',
 'LOUISIANA-14_JUL_20.csv',
 'JOHNSTON-WAKE-XMAS-30_APR_13.csv',
 'DELAWARE-19_DEC_18.csv',
 'NORTH_CAROLINA-30_JAN_17.csv',
 'VERMONT-30_APR_13.csv',
 'PUERTO_RICO-04_OCT_17.csv',
 'MARSHALL_ISLANDS-30_APR_13.csv',
 'MINNESOTA-30_JUN_18.csv',
 'KANSAS-27_JUN_19.csv',
 'CALIFORNIA-19_DEC_18.csv',
 'ALASKA-19_DEC_18.csv',
 'WISCONSIN-30_JAN_17.csv',
 'TEXAS-27_AUG_20.csv',
 'NEW_JERSEY-13_FEB_12.csv',
 'UTAH-23_MAY_20.csv',
 'OHIO-23_MAY_20.csv',
 'TENNESSEE-30_JUN_18.csv',
 'IDAHO-27_JUN_19.csv',
 'WYOMING-19_DEC_18.csv',
 'OREGON-27_AUG_20.csv',
 'GEORGIA-15_OCT_19.csv',
 'ALABAMA-14_SEP_16.csv',
 'MASSACHUSETTS-04_SEP_06.csv',
 'OKLAHOMA-15_OCT_19.csv',
 'VIRGINIA-22_NOV_14.csv',
 'SOUTH_CAROLINA-09_FEB_18.csv',
 'NORTH_DAKOTA-15_OCT_19.csv',
 'MISSISSIPPI-14_JUL_20.csv',
 'CONNECTICUT-09_SEP_08.csv',
 'ARIZONA-27_JUN_19.csv',
 'NEBRASKA-15_OCT_19.csv']

In [0]:
fout=open("us_stations_all.csv","a")

# now the rest:    
for file_ind, file_name in enumerate(us_station_filenames):
    with open("station_csvs/US_STATIONS/"+file_name) as f:
        for ind,line in enumerate(f):
            if ind == 0 and file_ind !=   0:
                pass
            else:
                fout.write(line)
fout.close()

### Verify the number of states is covered by FAT table

In [0]:
us_all = pd.read_csv("us_stations_all.csv")
us_all.head()

,CD,STATION,ICAO,IATA,SYNOP,LAT,LONG,ELEV,M,N,V,U,A,C,PRIOR,COUNTRY
0,MT,ALPHA,K1AM,1AM,NaN,47 17N,110 48W 12,28,X,NaN,NaN,NaN,NaN,NaN,9,US
1,MT,ANACONDA,K3U3,NaN,NaN,46 09N,112 52W 15,34,X,NaN,NaN,NaN,NaN,NaN,7,US
2,MT,BAKER,KBHK,NaN,NaN,46 21N,104 15W 9,4,X,NaN,NaN,NaN,A,NaN,7,US
3,MT,BIG TIMBER,K6S0,6S0,NaN,45 48N,109 58W 13,64,X,NaN,NaN,NaN,NaN,NaN,6,US
4,MT,BILLINGS,KBIL,BIL,72677.0,45 48N,108 33W 10,89,X,NaN,U,NaN,NaN,NaN,6,US


In [0]:
print(len(us_all["CD"].unique()))
us_all["CD"].unique()

56

array(['MT', 'PA', 'WA', 'NV', 'WV', 'KY', 'NY', 'GU', 'UM', 'SD', 'RI',
 'IL', 'HI', 'MI', 'IA', 'MD', 'NH', 'CO', 'AR', 'IN', 'DC', 'ME',
 'NM', 'FL', 'MO', 'LA', 'DE', 'NC', 'VT', 'PR', 'MH', nan, 'MN',
 'KS', 'CA', 'AK', 'WI', 'TX', 'NJ', 'UT', 'OH', 'TN', 'ID', 'WY',
 'OR', 'GA', 'AL', 'MA', 'OK', 'VA', 'SC', 'ND', 'MS', 'CT', 'AZ',
 'NE'], dtype=object)